In [76]:
import geemap
import geetools
import ee
#from PIL import Image
#from pycrc.CRCCCITT import CRCCCITT
#import numpy as np
#import matplotlib.pyplot as plt

In [77]:
ee.Authenticate()
ee.Initialize()

In [78]:
geemap.update_package()

Unzipping geemap-master.zip ...
Data downloaded to: C:\Users\nazin\Downloads\geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


In [79]:
Map = geemap.Map(center=(-14, 30), zoom=6)
Map

Map(center=[-14, 30], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

Loading a Shapefile

In [80]:
Shapefile_path = "D:/Land Use Land Cover Change/Study Sites Shapefile/Study_Sites_V1.shp"
Wards = geemap.shp_to_ee(Shapefile_path)
roi = Wards.filter(ee.Filter.eq('WardNm2020', 'LUUBWE'))
Map.addLayer(Wards, {}, "Shapefile")
Wards
roi

Image Collection

In [87]:
LUUBWE_sentinel = ee.ImageCollection('COPERNICUS/S2') \
.filterDate('2018-01-01', '2019-01-01') \
.filterBounds(Wards) \
.sort ('CLOUD_COVER') \
.median() \
.clip(Wards)
#.select('B[1-7]')
Map.addLayer(LUUBWE_sentinel, {'min': 0, 'max': 3000}, 'Land cover')

In [88]:
##LUUBWE_sentinel.get('CLOUD_COVER').getInfo()

Shapefile Collection

Cliping the Image and adding it to the Map 

In [89]:
rgbVis = {'bands':['B4','B3','B2'], min: 0, max: 0.3}
Map.addLayer(Wards, {'color': 'red'}, 'ROI')
Map.addLayer(LUUBWE_sentinel, rgbVis)
#Map.centerObject(LUUBWE_sentinel, 10)

Area of the shapefile

In [92]:
#Study_Sites_TotalArea =  Wards.geometry().area()
#Study_Sites_TotalArea_sqKm = ee.Number(Wards).divide(1e6)
#print(Study_Sites_TotalArea_sqKm)

Supervised classification

Merge all the training points

In [ ]:
var lable = "Class";
var sample = Waterbody.merge(Builtup).merge(Cropland).merge(Vegetation).merge(Bareland);
var bands = ['B2','B3','B4','B5','B6','B7'];
var training = LUUBWE_sentinel_clip.select(bands).sampleRegions({
  collection: sample, 
  properties: ['Class'], 
  scale: 30
});
print('training', training);

Training dataset divided in two ways
Traing the model: 80%
Test the model: 20% data for testing

In [ ]:
var dataset = training.randomColumn();
var trainset = dataset.filter(ee.Filter.lessThan('random', 0.8));
var testset = dataset.filter(ee.Filter.greaterThanOrEquals('random', 0.8));


Create the Model CART

In [ ]:
var classifier = ee.Classifier.smileCart().train(trainset, lable, bands);

Classify the sentinel image

In [ ]:
var classified = LUUBWE_sentinel_clip.classify(classifier);
print('classified', classified);

Visualisation of the classification

In [ ]:
var landcoverpalette = [
  'blue',
  'red',
  'yellow',
  'green',
  'grey',
  ];
  
Map.addLayer(classified.clip(roi),{'min': 0, 'max': 4, 'palette': landcoverpalette});

Get information about the trained classifier.

In [ ]:
print('Results of trained classifier', classifier.explain());

Acuracy assement

Get a confusion matrix and overall accuracy for the training sample.

In [ ]:
var trainAccuracy = classifier.confusionMatrix();
print('Training error matrix', trainAccuracy);
print('Training overall accuracy', trainAccuracy.accuracy());

Get a confusion matrix and overall accuracy for the validation sample.

In [ ]:
testset = trainset.classify(classifier);
var validationAccuracy = testset.errorMatrix(lable, 'classification');
print('Validation error matrix', validationAccuracy);
print('Validation accuracy', validationAccuracy.accuracy());

Calculating Area by Class by Reagion
Calculating the Nested Group area of classes

In [ ]:
var areaImage = ee.Image.pixelArea().addBands(classified);
var areas = areaImage.reduceRegion({
  reducer: ee.Reducer.sum().group({
    groupField:1,
    groupName: 'class'}), 
    geometry: roi,
    scale: 10,
    maxPixels: 1e13});

print(areaImage);

Flatening the Nested Gropus areas of classes

In [ ]:
var classAreas = ee.List(areas.get('groups'));
var classAreaLists = classAreas.map(function(item){
  var areaDic = ee.Dictionary(item)
  var classNumber = ee.Number(areaDic.get('class')).format()
  var area = ee.Number(areaDic.get('sum')).divide(1e6).round()
  return ee.List([classNumber, area])
});

var results = ee.Dictionary(classAreaLists.flatten());
print(results);

Export Image

In [ ]:
Export.image.toDrive({
image:classified,
description: 'LUUBWE_sentinel_classified_2019',
fileFormat: 'GeoTIFF',
scale: 10,
region: roi,
});

Export table to your google drive

In [ ]:
Export.table.toDrive({
        collection: results,
        description: 'classAreaLists', 
        folder: 'EE',
    fileNamePrefix: 'classAreas', 
    fileFormat: 'CSV'
    });